In [1]:
import pandas as pd
import numpy as np
import gc; gc.enable()
import lightgbm as lgb

from GridSearcher import data_loader, model_loader, fit_params, get_oof_predictions

In [2]:
DATA_TYPE = 'char_wb'
DATA_NORMALIZE = True
IS_REGRESSION = True
SEED = 411

## Do SVD

In [3]:
dl = data_loader(data_type=DATA_TYPE, is_regression=IS_REGRESSION, is_train=True)
all_X, train_y = dl.load()
train_len = train_y.shape[0]

Arguments: char_wb True True
target loaded
title text features loaded
param text features loaded
desc text features loaded


In [4]:
type(all_X), all_X.shape

(scipy.sparse.csr.csr_matrix, (2011862, 279422))

In [5]:
from sklearn.decomposition import TruncatedSVD
# TruncatedSVD(n_components=2, algorithm=’randomized’, n_iter=5, random_state=None, tol=0.0)[source]

In [6]:
tsvd = TruncatedSVD(n_components=300, algorithm='arpack', random_state=411719)
all_X_svd = tsvd.fit_transform(all_X)
tsvd.explained_variance_

array([ 439.92229498,  539.9118715 ,  119.56263095,  116.86387758,
         87.49059965,   54.64663454,   61.3533066 ,   50.58013541,
         38.39164954,   23.76596547,   20.06487546,   17.70922937,
         13.85358199,   13.42427476,   12.81950546,   11.64736647,
         11.26427776,   10.74976085,   10.11096272,    9.5940215 ,
          9.17577401,    9.11276205,    8.93381965,    8.05373599,
          7.97188703,    7.57201182,    7.29481739,    7.13008387,
          6.98169407,    6.75591725,    6.37865379,    6.15604852,
          6.02305709,    5.98507301,    5.79331676,    5.68024373,
          5.49964375,    5.44060774,    5.33145532,    5.22189281,
          5.06652638,    4.98962628,    4.87028585,    4.72379757,
          4.6189782 ,    4.59157632,    4.46132768,    4.44384904,
          4.35478362,    4.32257323,    4.24344955,    4.14077869,
          4.10212901,    4.00211667,    3.9125964 ,    3.81598344,
          3.77663836,    3.70501729,    3.66300868,    3.58638

In [7]:
import pickle
with open('desc_text_feature_count_vec_svd.pickle', 'wb') as handle:
    pickle.dump(all_X_svd, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
del all_X; gc.collect()

103

In [9]:
all_X = all_X_svd
all_X.shape

(2011862, 300)

In [10]:
all_X[:3,:]

array([[  1.16122546e+01,  -2.89847943e+00,  -3.12846150e+00,
         -4.45546972e-01,  -9.71034068e-01,   5.85545872e+00,
         -9.80929341e+00,   1.11792024e+00,   3.27761923e+00,
         -1.50003049e+00,   5.22990102e+00,   2.00384192e+00,
         -1.28915485e+00,  -2.87460458e+00,   1.29216024e+00,
          1.31841240e+00,  -1.72102583e+00,   2.20059611e+00,
          4.56534000e-01,   2.83590318e-01,  -1.81320834e-01,
          3.46646492e-01,  -2.59403411e+00,  -4.29973660e-01,
         -1.52674381e+00,  -5.86611809e-01,  -1.57558836e+00,
         -2.19073013e+00,   2.65921959e+00,   7.22223555e-01,
          4.66248887e-01,   1.38022016e+00,   1.25170938e+00,
         -1.38258399e+00,  -3.05221519e-01,   6.44493729e-01,
         -1.45569861e+00,   8.24670554e-01,   1.97740784e+00,
          1.89766522e+00,  -4.14900478e+00,   2.38228113e+00,
         -1.57181654e+00,  -1.40632241e-01,  -1.96255903e+00,
          4.96503890e-01,  -3.37106563e+00,   1.06837592e+00,
        

## Load from SVD data

In [3]:
if IS_REGRESSION:
    train_y = pd.read_csv("regression_target.csv")
else:
    train_y = pd.read_csv("classification_target.csv")

train_y = train_y['deal_probability'].values
train_len = train_y.shape[0]

In [4]:
import pickle
with open('desc_text_feature_count_vec_svd.pickle', 'rb') as handle:
    all_X = pickle.load(handle)

## Ridge

In [5]:
ml = model_loader(model_type='rg')

In [12]:
default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

try_params = \
[
    {
        'alpha': [1., 2., 4., 8., 16.],
    }
]

fit_params(all_X[:len(train_y)], train_y, ml, default_params, try_params, fit_params=None, seed=SEED)

{'alpha': 1.0} train loss: 0.238570, valid loss:0.239483, loss_diff:0.000914
{'alpha': 1.0} train loss: 0.238729, valid loss:0.238821, loss_diff:0.000092
{'alpha': 1.0} train loss: 0.238791, valid loss:0.238608, loss_diff:-0.000183
{'alpha': 1.0} train loss: 0.238692, valid loss:0.239003, loss_diff:0.000311
{'alpha': 1.0} train loss: 0.238914, valid loss:0.238088, loss_diff:-0.000826
=================>{'alpha': 1.0} loss:0.238801
{'alpha': 2.0} train loss: 0.238570, valid loss:0.239483, loss_diff:0.000914
{'alpha': 2.0} train loss: 0.238729, valid loss:0.238821, loss_diff:0.000092
{'alpha': 2.0} train loss: 0.238791, valid loss:0.238608, loss_diff:-0.000183
{'alpha': 2.0} train loss: 0.238692, valid loss:0.239003, loss_diff:0.000311
{'alpha': 2.0} train loss: 0.238914, valid loss:0.238088, loss_diff:-0.000826
=================>{'alpha': 2.0} loss:0.238801
{'alpha': 4.0} train loss: 0.238570, valid loss:0.239483, loss_diff:0.000914
{'alpha': 4.0} train loss: 0.238729, valid loss:0.23882

,param,val_loss_mean,val_loss_std
0,{'alpha': 1.0},0.238801,0.000459
1,{'alpha': 2.0},0.238801,0.000459
2,{'alpha': 4.0},0.238801,0.000459
3,{'alpha': 8.0},0.238801,0.000459
4,{'alpha': 16.0},0.238801,0.000459


In [13]:
{'alpha': 1.0} train loss: 0.238408, valid loss:0.239318, loss_diff:0.000910
{'alpha': 1.0} train loss: 0.238569, valid loss:0.238645, loss_diff:0.000075
{'alpha': 1.0} train loss: 0.238627, valid loss:0.238442, loss_diff:-0.000185
{'alpha': 1.0} train loss: 0.238528, valid loss:0.238837, loss_diff:0.000309
{'alpha': 1.0} train loss: 0.238747, valid loss:0.237943, loss_diff:-0.000804

SyntaxError: invalid syntax (<ipython-input-13-0a2f4148409a>, line 1)

In [6]:
default_params = {
    'alpha': 1.0, 
    'fit_intercept': True, 
    'normalize': False, 
    'copy_X': True, 
    'max_iter': None, 
    'tol': 0.001, 
    'solver':'auto', 
    'random_state': SEED
}

fit_params = None

rg_oof_val_pred, rg_oof_test_pred, models = get_oof_predictions(all_X[:len(train_y)], train_y, all_X[len(train_y):], ml, 
                                                                  default_params, seed=SEED, fit_params=fit_params, 
                                                                  use_eval_set=False)

Fold 1 completed.
Fold 2 completed.
Fold 3 completed.
Fold 4 completed.
Fold 5 completed.


In [7]:
pd.DataFrame(data=rg_oof_val_pred, columns=['text_cwb_rg_pred']).to_csv('text_cwb_rg_oof_val_pred.csv', index=False)
pd.DataFrame(data=rg_oof_test_pred, columns=['text_cwb_rg_pred']).to_csv('text_cwb_rg_oof_test_pred.csv', index=False)